**Importing Dependencies**

In [5]:
import matplotlib.pyplot as plt
import pathlib
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from google.colab import files
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

**Downloading dataset**

In [ ]:
#Loading the dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file("flower_photos", origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)
fashion_mnist = keras.datasets.fashion_mnist

#Splitting the dataset into train and test images
(train_images, train_lables), (evaluating_images, evaluating_labels) = tf.keras.datasets.fashion_mnist.load_data()
train_images_reshaped = train_images.reshape(60000, 28, 28, 1)

**Functions for the task**

In [3]:
#Function the inform user about the task
def task_information():
  print("Greetings of the day..!!!\
  This task usualyy focuses around the training of various models and saving all the necessary details of all the models\
  into csv files. The quantity of total number of models depends on user's choice and users are allowed to tune hyperparameter\
  at every layer of the individual model.  ")

#Creating the model
def build_model(num_layers, model_number, input_shape=(28, 28, 1)):
  
  #Necessary parameters for building the model
  i = 1
  filters = []
  kernel_sizes = []
  activations = []
  pooling_sizes = []
  paddings = []

  epochs = int(input("Enter the number of epochs for Model {}: ".format(model_number)))
  print("")

  #Loop to build the number of layers of the model, #Get idea from my cousin
  while i < num_layers+1:
    filter_value = int(input("Enter filter size for {} layer: ".format(i)))
    filters.append(filter_value)

    kernel_size_value = int(input("Enter kernel size for {} layer: ".format(i)))
    kernel_sizes.append(kernel_size_value)

    activation_value = input("Enter activation function for {} layer: ".format(i))
    activations.append(activation_value)

    pooling_size_value = int(input("Enter pooling size for {} layer: ".format(i)))
    pooling_sizes.append(pooling_size_value)

    padding_value = input("Enter the padding value for {} layer: ".format(i))
    paddings.append(padding_value)

    print()
    i += 1

  #Using sequential as base model for the neural network
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape))

  #Using zip function to implement crucial parameters in the layers, #Get idea from my cousin
  for filter, kernel_size, activation, pool_size, padding, i in zip(filters, kernel_sizes, activations, pooling_sizes, paddings, range(1, num_layers+1)):
      model.add(Conv2D(filters=filter, kernel_size=(kernel_size,kernel_size), activation=activation, padding=padding))
      model.add(MaxPooling2D(pool_size=(pool_size,pool_size)))

  model.add(Flatten())
  model.add(Dense(units=10, activation='softmax'))
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(train_images_reshaped, train_lables, epochs = epochs, validation_split = 0.4)
  return history, model, epochs

#Accumulating all the functions in one main function
def main():
  #Introduction of the task
  task_information()
  print("")

  number_models = int(input("Please enter the number of models: "))
  print("")

  dataframe = pd.DataFrame(columns=['Model Name', 'Architecture' ,'Training Accuracy', 'Validation Accuracy', 'Training Loss', 'Validation Loss', 'Epochs'])

  i = 1
  #Loop used to ask user to put parameters for each layers
  while i < number_models+1:
    summary = []
    print("")
    print("Enter the details of achitecture and hyperparameters for Model {}".format(i))
    layers = int(input("Enter the layers for Model {}: ".format(i)))
    print("")
    history, model, epochs = build_model(layers, i) #Calling build model to return hyperparameters and history of model
    model.summary(print_fn=lambda x: summary.append(x)) #Using lambda function that will take number of arguments and returns only one expression #Get help from my cousin
    dataframe.loc[i] = [model.name, summary[2:], max(history.history['accuracy']), max(history.history['val_accuracy']), min(history.history['loss']), min(history.history['val_loss']), epochs]
    i += 1

  #Saving the dataframe of information of all the models
  print(dataframe)
  dataframe.to_csv('output.csv', encoding = 'utf-8-sig', index=False) 
  files.download('output.csv')


In [ ]:
if __name__ == "__main__":
  main()